# Motion across sessions - comparisson with the cemented dataset

In [ ]:
from labdata.schema import *
import pylab as plt
%matplotlib widget
from labdata import chronic_paper as paper
plt.matplotlib.rcParams['pdf.fonttype'] = 42
plt.matplotlib.rcParams['ps.fonttype'] = 42
savepath = Path('../../figures/figure3')
savepath.mkdir(parents = True,exist_ok = True)


In [ ]:
%matplotlib widget
key = dict(subject_name="_AL031",shank_num=0)
import pylab as plt
plt.figure()
paper.ConcatenatedSpikes().plot_raster(key, corrected=False, overlay_dredge=True,cmap = 'gray')
plt.axis('tight')
plt.ylim(4200, 5200)

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedRecordings()*Session() & 'subject_name = "_AL031"'& 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0]))
intervals_days_cemented = intervals_days.astype('timedelta64[D]') # these are the intervals for which we have _AL036 data


# selected JC131 sessions
intervals_days = (paper.ConcatenatedSpikes.IncludedRecordings()*Session() & 'subject_name = "JC131"'& 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')
arg_sessions = []
for i in intervals_days_cemented:
    arg_sessions.append(np.where(intervals_days<=i)[0][-1])
arg_sessions
intervals_days,intervals_days_cemented,arg_sessions

In [ ]:
data_AL036 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "_AL036"' & 'shank_num = 0').fetch1()
intervals_days = (paper.ConcatenatedSpikes.IncludedRecordings()*Session() & 'subject_name = "_AL036"'& 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0]))
intervals_days_cemented = intervals_days.astype('timedelta64[D]') # these are the intervals for which we have _AL036 data

data_AL036['spike_times_s'] = data_AL036['spike_times_s'].astype(np.float32)
displ = data_AL036['displacement']
sp= data_AL036['spike_times_s']

from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(sp,data_AL036['spike_depths_um'],data_AL036['spike_amps'],cmap = 'gray_r',clim = [0,300])

plt.vlines(offsets,3500,4500,'k',lw = 0.5)
plt.ylim([3500,4500])
plt.plot(displ+np.mean([3500,4500]),'r')
offsets = np.hstack([data_AL036['session_breaks'],np.max(sp)])
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days_cemented.astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('AL036 cortex (shank 0)')
plt.savefig(savepath/f'AL036_shank0_drift.pdf')

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedRecordings()*Session() & 'subject_name = "JC131"'& 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')

idx_JC131 = [0,1,8,30,34,46,59,68,76,77,78,84,87]

data_JC131 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "JC131"' & 'shank_num = 0').fetch1()

data_JC131['spike_times_s'] = data_JC131['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([4,data_JC131['session_breaks']]),
                          np.hstack([data_JC131['session_breaks'],np.max(data_JC131['spike_times_s'])])]).T[idx_JC131]
t = data_JC131['time_bin_centers_s']
displ = data_JC131['displacement'][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_JC131['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(np.hstack(nsp),data_JC131['spike_depths_um'][idx],data_JC131['spike_amps'][idx],cmap = 'gray_r',clim = [0,300])

plt.vlines(offsets,4200,5200,'k',lw = 0.5)
plt.ylim([4200,5200])
plt.plot(displ+np.mean([4200,5200]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_JC131].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('JC131 cortex (shank 0)')
plt.savefig(savepath/f'JC131_shank0_drift.pdf')

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedRecordings()*Session() & 'subject_name = "JC131"'& 'shank_num = 2').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')

idx_JC131 = [0,1,8,30,34,46,59,68,76,77,78,84,87]

data_JC131 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "JC131"' & 'shank_num = 2').fetch1()

data_JC131['spike_times_s'] = data_JC131['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([0,data_JC131['session_breaks']]),
                          np.hstack([data_JC131['session_breaks'],np.max(data_JC131['spike_times_s'])])]).T[idx_JC131]
t = data_JC131['time_bin_centers_s']
displ = data_JC131['displacement'][2][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_JC131['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(np.hstack(nsp),data_JC131['spike_depths_um'][idx],data_JC131['spike_amps'][idx],cmap = 'gray_r',clim = [0,300])

plt.vlines(offsets,1300,2300,'k',lw = 0.5)
plt.ylim([1300,2300])
plt.plot(displ+np.mean([1300,2300]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_JC131].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('JC131 thalamus (shank 2)')
plt.savefig(savepath/f'JC131_shank2_drift.pdf')

In [ ]:
intervals_days = (paper.ConcatenatedSpikes.IncludedRecordings()*Session() & 'subject_name = "MM012"'&
                  'probe_num = 2' & 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')
idx_MM012 = [0,1,6,18, 19, 20,24,32, 33, 40, 41 ,44,48]
data_MM012 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "MM012"' 
              & 'probe_num = 2' 
              & 'shank_num = 0').fetch1()


data_MM012['spike_times_s'] = data_MM012['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([4,data_MM012['session_breaks']]),
                          np.hstack([data_MM012['session_breaks'],np.max(data_MM012['spike_times_s'])])]).T[idx_MM012]
t = data_MM012['time_bin_centers_s']
displ = data_MM012['displacement'][2][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_MM012['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(np.hstack(nsp),data_MM012['spike_depths_um'][idx],data_MM012['spike_amps'][idx],cmap = 'gray_r',clim = [0,70])

plt.vlines(offsets,2800,3800,'k',lw = 0.5)
plt.ylim([2800,3800])
plt.plot(displ+np.mean([2800,3800]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_MM012].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('MM012 cortex (NP1)')
plt.savefig(savepath/f'MM012_shank2_drift.pdf')

In [ ]:

intervals_days = (paper.ConcatenatedSpikes.IncludedRecordings()*Session() & 'subject_name = "MM012"'&
                  'probe_num = 1' & 'shank_num = 0').fetch('session_datetime')
intervals_days = (intervals_days.astype(np.datetime64) - np.datetime64(intervals_days[0])).astype('timedelta64[D]')
idx_MM012 = [0,1,6,18, 19, 20,24,32, 33, 40, 41 ,44,48]
data_MM012 = (paper.ConcatenatedSpikes*paper.ConcatenatedSpikes.DredgeResults & 'subject_name = "MM012"' 
              & 'probe_num = 1' 
              & 'shank_num = 0').fetch1()


data_MM012['spike_times_s'] = data_MM012['spike_times_s'].astype(np.float32)
sessiontimes = np.vstack([np.hstack([4,data_MM012['session_breaks']]),
                          np.hstack([data_MM012['session_breaks'],np.max(data_MM012['spike_times_s'])])]).T[idx_MM012]
t = data_MM012['time_bin_centers_s']
displ = data_MM012['displacement'][-2][np.hstack([np.where((t>a) & (t<=b))[0] for a,b in sessiontimes])]
sp= data_MM012['spike_times_s']
idx = [np.where((sp>a) & (sp<=b))[0] for a,b in sessiontimes]

nsp = []
offsets = []
offset = 0
for i in idx:
    nsp.append(sp[i]-sp[i][0]+offset)
    offset = nsp[-1][-1]
    offsets.append(offset)
idx = np.hstack(idx)
from spks.viz import plot_drift_raster

plt.figure(figsize = [7,3])
plot_drift_raster(np.hstack(nsp),data_MM012['spike_depths_um'][idx],data_MM012['spike_amps'][idx],cmap = 'gray_r',clim = [0,70])

plt.vlines(offsets,2600,3600,'k',lw = 0.5)
plt.ylim([2600,3600])
plt.plot(displ+np.mean([2600,3600]),'r')
plt.xticks(offsets - np.mean(np.diff(offsets))/2,[a for a in intervals_days[idx_MM012].astype(int)],rotation = 0,fontsize = 8);
plt.yticks(fontsize = 8)
plt.ylabel("Probe location (um)")
plt.xlabel("Days from first recording");
plt.title('MM012 motor cortex (NP1)')
plt.savefig(savepath/f'MM012_shank0_drift.pdf')